In [186]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

In [187]:
data = pd.read_csv('Employee.csv')
data

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1
...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4649,Masters,2013,Pune,2,37,Male,No,2,1
4650,Masters,2018,New Delhi,3,27,Male,No,5,1
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


In [188]:
data.describe()

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,LeaveOrNot
count,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000
mean,2015.062970,2.698259,29.393295,2.905652,0.343864
std,1.863377,0.561435,4.826087,1.558240,0.475047
min,2012.000000,1.000000,22.000000,0.000000,0.000000
25%,2013.000000,3.000000,26.000000,2.000000,0.000000
50%,2015.000000,3.000000,28.000000,3.000000,0.000000
75%,2017.000000,3.000000,32.000000,4.000000,1.000000
max,2018.000000,3.000000,41.000000,7.000000,1.000000


In [189]:
data['LeaveOrNot'].value_counts()

0    3053
1    1600
Name: LeaveOrNot, dtype: int64

In [190]:
duplicates_rows = data[data.duplicated()]
print ("Duplicated rows are :")
duplicates_rows

Duplicated rows are :


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
111,Bachelors,2017,Pune,2,27,Female,No,5,1
130,Bachelors,2017,Bangalore,3,26,Female,No,4,0
138,Bachelors,2017,New Delhi,3,28,Male,No,2,0
160,Bachelors,2014,Bangalore,3,28,Female,No,3,0
167,Bachelors,2014,Bangalore,3,25,Male,No,3,0
...,...,...,...,...,...,...,...,...,...
4640,Bachelors,2015,Bangalore,3,35,Male,No,0,0
4642,Bachelors,2012,Bangalore,3,36,Female,No,4,0
4646,Bachelors,2013,Bangalore,3,25,Female,No,3,0
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0


In [191]:
duplicate_count = data[data.duplicated()].shape[0]

# Print the count of duplicate rows
print("Number of duplicate rows:", duplicate_count)

Number of duplicate rows: 1889


In [192]:
# Remove duplicate rows and update 'data'
data = data.drop_duplicates()

# If you want to reset the index after removing duplicates
data.reset_index(drop=True, inplace=True)

In [193]:
duplicates_rows = data[data.duplicated()]
print ("Duplicated rows are :")
duplicates_rows

Duplicated rows are :


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot


In [194]:
data = pd.get_dummies(data,drop_first=True)

In [195]:
X = data.drop(columns=['LeaveOrNot'])
y = data['LeaveOrNot']

In [196]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [197]:
# Automatically split columns into categorical and numeric
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numeric_cols = X.select_dtypes(exclude=['object']).columns.tolist()

In [198]:
# Create transformers
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

In [199]:
# Combine transformers
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_cols),('cat', categorical_transformer, categorical_cols)] ,remainder='passthrough')

In [200]:
# Preprocess the data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [201]:
# Define and compile your DNN model with regularization and dropout
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [202]:
# Use a learning rate schedule to gradually reduce the learning rate
initial_learning_rate = 0.01
lr_schedule = LearningRateScheduler(lambda epoch: initial_learning_rate / (10 ** (epoch // 10)))

In [203]:
# Compile the model with the Adam optimizer and binary cross-entropy loss
model.compile(optimizer=Adam(learning_rate=initial_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

In [204]:
# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

In [205]:
# Train the model with early stopping based on validation loss
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[lr_schedule, early_stopping])

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

Epoch 1/50
56/56 [==============================] - 1s 7ms/step - loss: 0.9687 - accuracy: 0.6176 - val_loss: 0.6646 - val_accuracy: 0.7291 - lr: 0.0100
Epoch 2/50
56/56 [==============================] - 0s 4ms/step - loss: 0.6718 - accuracy: 0.6736 - val_loss: 0.6286 - val_accuracy: 0.7314 - lr: 0.0100
Epoch 3/50
56/56 [==============================] - 0s 4ms/step - loss: 0.6734 - accuracy: 0.6787 - val_loss: 0.6130 - val_accuracy: 0.7449 - lr: 0.0100
Epoch 4/50
56/56 [==============================] - 0s 3ms/step - loss: 0.6517 - accuracy: 0.6912 - val_loss: 0.6253 - val_accuracy: 0.7201 - lr: 0.0100
Epoch 5/50
56/56 [==============================] - 0s 4ms/step - loss: 0.6587 - accuracy: 0.6912 - val_loss: 0.6073 - val_accuracy: 0.7562 - lr: 0.0100
Epoch 6/50
56/56 [==============================] - 0s 3ms/step - loss: 0.6463 - accuracy: 0.7014 - val_loss: 0.5945 - val_accuracy: 0.7517 - lr: 0.0100
Epoch 7/50
56/56 [==============================] - 0s 3ms/step - loss: 0.6386 - a

In [206]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy: {accuracy:.2f}')

# Print classification report for more detailed evaluation
print(classification_report(y_test, y_pred_binary))

Accuracy: 0.79
              precision    recall  f1-score   support

           0       0.77      0.93      0.85       333
           1       0.85      0.58      0.69       220

    accuracy                           0.79       553
   macro avg       0.81      0.76      0.77       553
weighted avg       0.80      0.79      0.78       553

